In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import random
from collections import Counter
import itertools
import networkx as nx
import ast
import math

In [ ]:
def centeroidnp(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    sum_z = np.sum(arr[:, 2])
    return (sum_x/length, sum_y/length, sum_z/length)

In [ ]:
def split(word):
    return [char for char in word]

In [313]:
# Using readlines()
file1 = open('aoc-19', 'r')
file1 = open('aoc-19-ex2', 'r')
file1 = open('aoc-19-ex', 'r')
Lines = file1.readlines()
Lines = [x.replace('\n','') for x in Lines]
scanners = []
helper = []
for ele in Lines:
    if ele.find('scanner') > 0 :
        scanners.append(np.array(helper))
        helper = []
    else:
        xo = ele.split(',')
        helper.append([int(xo[0]),int(xo[1]),int(xo[2])])
scanners.append(np.array(helper))
print(len(scanners[1]),len(scanners[2]))
print(scanners[1])

25 25
[[ 404 -588 -901]
 [ 528 -643  409]
 [-838  591  734]
 [ 390 -675 -793]
 [-537 -823 -458]
 [-485 -357  347]
 [-345 -311  381]
 [-661 -816 -575]
 [-876  649  763]
 [-618 -824 -621]
 [ 553  345 -567]
 [ 474  580  667]
 [-447 -329  318]
 [-584  868 -557]
 [ 544 -627 -890]
 [ 564  392 -477]
 [ 455  729  728]
 [-892  524  684]
 [-689  845 -530]
 [ 423 -701  434]
 [   7  -33  -71]
 [ 630  319 -379]
 [ 443  580  662]
 [-789  900 -551]
 [ 459 -707  401]]


In [315]:
all(np.array([553,889,-390])) in scanners[1]

False

In [ ]:
asdf = [np.array([1,1,1])]
print(all(np.array([1,1,1])) in asdf)

In [226]:
dist1 = {}
dist_all = []
for x in scanners[1]:
    #if np.array_equal(x,[-618,-824,-621]):
    for y in scanners[1]:
        if not np.array_equal(x,y):
            #print(x,y,abs(x-y))
            dist1[str(abs(x-y))] = (x,y) #.append(((x,y),str(abs(x-y))))
            dist_all.append(str(abs(x-y)))

dist2 = {}
for x in scanners[2]:
    #if np.array_equal(x,[686,422,578]):
    for y in scanners[2]:
        if not np.array_equal(x,y):
            #print(x,y,abs(x-y))
            dist_all.append(str(abs(x-y)))
            dist2[str(abs(x-y))] = (x,y)
#
##cnt = Counter(dist_all)
##print(cnt.most_common(12))
#print(dist_all)

In [254]:
cnt = Counter(dist_all)
fours = []
for ele in cnt:
    if cnt[ele] == 4:
        fours.append(ele)
#print(set(fours),len(set(fours)))
final_points = []
for ele in set(fours):
    #print(ele, dist1[ele],dist2[ele])
    final_points.append(str(dist1[ele][0]))
    final_points.append(str(dist1[ele][1]))
    final_points.append(str(dist2[ele][0]))
    final_points.append(str(dist2[ele][1]))



In [269]:
def strToArray(input):
    helper = re.findall('\-*\d+',input)
    #print(helper)
    return np.array([int(helper[0]),int(helper[1]),int(helper[2])])

In [306]:
len(np.unique(final_points))

24

In [321]:
def elementInList(element,array):
    for ele in array:
        if np.array_equal(ele,element):
            return True
    return False

In [324]:
final_points[0]

'[-345 -311  381]'

In [326]:
elementInList(strToArray(final_points[0]),scanners[1])

True

In [327]:
V0 = []
V1 = []
print(len(np.unique(final_points)))
for ele in np.unique(final_points):
    if elementInList(strToArray(ele),scanners[1]):
        V0.append(strToArray(ele))
    if elementInList(strToArray(ele),scanners[2]):
        V1.append(strToArray(ele))        
V0 = np.matrix(V0)
V1 = np.matrix(V1)

24


In [329]:
print(len(V0),len(V1))

12 12


In [302]:
strToArray('[-355,  545, -477]') in scanners[2]

True

In [336]:
A =np.matrix([[-618,-824,-621],
[-537,-823,-458],
[-447,-329,318],
[404,-588,-901],
[544,-627,-890],
[528,-643,409],
[-661,-816,-575],
[390,-675,-793],
[423,-701,434],
[-345,-311,381],
[459,-707,401],
[-485,-357,347]])

B = np.matrix([[686,422,578],
[605,423,415],
[515,917,-361],
[-336,658,858],
[-476,619,847],
[-460,603,-452],
[729,430,532],
[-322,571,750],
[-355,545,-477],
[413,935,-424],
[-391,539,-444],
[553,889,-390]])

In [289]:
# Input: expects 3xN matrix of points
# Returns R,t
# R = 3x3 rotation matrix
# t = 3x1 column vector

def rigid_transform_3D(A, B):
    assert A.shape == B.shape

    num_rows, num_cols = A.shape
    if num_rows != 3:
        raise Exception(f"matrix A is not 3xN, it is {num_rows}x{num_cols}")

    num_rows, num_cols = B.shape
    if num_rows != 3:
        raise Exception(f"matrix B is not 3xN, it is {num_rows}x{num_cols}")

    # find mean column wise
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)

    # ensure centroids are 3x1
    centroid_A = centroid_A.reshape(-1, 1)
    centroid_B = centroid_B.reshape(-1, 1)

    # subtract mean
    Am = A - centroid_A
    Bm = B - centroid_B

    H = Am @ np.transpose(Bm)

    # sanity check
    #if linalg.matrix_rank(H) < 3:
    #    raise ValueError("rank of H = {}, expecting 3".format(linalg.matrix_rank(H)))

    # find rotation
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("det(R) < R, reflection detected!, correcting for it ...")
        Vt[2,:] *= -1
        R = Vt.T @ U.T

    t = -R @ centroid_A + centroid_B

    return R, t

In [340]:
A.H

matrix([[-618, -537, -447,  404,  544,  528, -661,  390,  423, -345,
          459, -485],
        [-824, -823, -329, -588, -627, -643, -816, -675, -701, -311,
         -707, -357],
        [-621, -458,  318, -901, -890,  409, -575, -793,  434,  381,
          401,  347]])

In [339]:
V0.H

matrix([[ 390,  404,  423,  459,  528,  544, -345, -447, -485, -537,
         -618, -661],
        [-675, -588, -701, -707, -643, -627, -311, -329, -357, -823,
         -824, -816],
        [-793, -901,  434,  401,  409, -890,  381,  318,  347, -458,
         -621, -575]])

In [337]:
ret_R, ret_t = rigid_transform_3D(A.H,B.H)
print(ret_R,ret_t)

[[-1.00000000e+00 -1.38777878e-17 -3.29597460e-17]
 [ 0.00000000e+00  1.00000000e+00  2.22044605e-16]
 [ 1.38777878e-17  1.94289029e-16 -1.00000000e+00]] [[  68.]
 [1246.]
 [ -43.]]


In [341]:
[0,0,0] * ret_R + ret_t

NameError: name 'ret_r' is not defined

In [ ]:
C = np.matrix([[459,-707,401],
[-739,-1745,668],
[-485,-357,347],
[432,-2009,850],
[528,-643,409],
[423,-701,434],
[-345,-311,381],
[408,-1815,803],
[534,-1912,768],
[-687,-1600,576],
[-447,-329,318],
[-635,-1737,486]])

In [293]:
V1.shape

(12, 3)

[[-0.25925617 -0.52762642 -0.80894783]
 [ 0.82372971 -0.55809171  0.100015  ]
 [-0.50423764 -0.64042486  0.57931028]] [[-367.43640727]
 [ 324.96493319]
 [-196.10416212]]
